# Exploratory Data Analysis

Ce notebook présente une analyse descriptive du jeu de données final construit dans `0_data_collection_cleaning.ipynb`.
On explore la distribution des variables, les valeurs extrêmes, et les relations entre pollution et niveau de vie.


In [31]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("../data")
PROC_DIR = DATA_DIR / "processed"

df = pd.read_csv(PROC_DIR / "df_final.csv")
df.head()


,Commune,pm10,pm25,no2,Commune_clean,CODGEO_x,CODGEO_y,CODGEO,NBPERS19,Q219,D119,GI19,poverty_index
0,ABBEVILLE LA RIVIERE,15.0,8.0,8.0,ABBEVILLE LA RIVIERE,91001,91001,91001,336,24590,NaN,NaN,NaN
1,ABLEIGES,18.0,9.0,13.0,ABLEIGES,95002,95002,95002,1153,28270,NaN,NaN,NaN
2,ABLIS,15.0,8.0,13.0,ABLIS,78003,78003,78003,3563,25970,16030.0,0.208,0.617251
3,ABLON SUR SEINE,17.0,10.0,24.0,ABLON SUR SEINE,94001,94001,94001,5741,22780,11530.0,0.270,0.506146
4,ACHERES,19.0,10.0,21.0,ACHERES,78005,78005,78005,22032,22260,12030.0,0.257,0.540431


## Step 1 — Vue d'ensemble + qualité des données





In [32]:
df.shape, df.columns
df.head(5)

,Commune,pm10,pm25,no2,Commune_clean,CODGEO_x,CODGEO_y,CODGEO,NBPERS19,Q219,D119,GI19,poverty_index
0,ABBEVILLE LA RIVIERE,15.0,8.0,8.0,ABBEVILLE LA RIVIERE,91001,91001,91001,336,24590,NaN,NaN,NaN
1,ABLEIGES,18.0,9.0,13.0,ABLEIGES,95002,95002,95002,1153,28270,NaN,NaN,NaN
2,ABLIS,15.0,8.0,13.0,ABLIS,78003,78003,78003,3563,25970,16030.0,0.208,0.617251
3,ABLON SUR SEINE,17.0,10.0,24.0,ABLON SUR SEINE,94001,94001,94001,5741,22780,11530.0,0.270,0.506146
4,ACHERES,19.0,10.0,21.0,ACHERES,78005,78005,78005,22032,22260,12030.0,0.257,0.540431


In [33]:
df.describe(include="all").T.head(20)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Commune,1125,1125,CHAMPAGNE SUR SEINE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pm10,1120.0,NaN,NaN,NaN,16.695536,2.378256,14.0,15.0,16.0,18.0,27.0
pm25,1121.0,NaN,NaN,NaN,9.03479,1.093668,8.0,8.0,9.0,9.0,15.0
no2,1122.0,NaN,NaN,NaN,14.209447,7.729583,5.0,8.0,13.0,18.0,49.0
Commune_clean,1125,1125,CHAMPAGNE SUR SEINE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CODGEO_x,1125.0,NaN,NaN,NaN,83735.801778,7717.426505,77001.0,77321.0,78314.0,91634.0,95682.0
CODGEO_y,1125.0,NaN,NaN,NaN,83735.801778,7717.426505,77001.0,77321.0,78314.0,91634.0,95682.0
CODGEO,1125.0,NaN,NaN,NaN,83735.801778,7717.426505,77001.0,77321.0,78314.0,91634.0,95682.0
NBPERS19,1125.0,NaN,NaN,NaN,7827.003556,15411.023147,101.0,561.0,1356.0,6718.0,117954.0
Q219,1125.0,NaN,NaN,NaN,26249.048889,4179.662577,13960.0,23800.0,26010.0,28280.0,46180.0


In [34]:
(df.isna().mean().sort_values(ascending=False) * 100).head(15)


D119             57.333333
GI19             57.333333
poverty_index    57.333333
pm10              0.444444
pm25              0.355556
no2               0.266667
Commune           0.000000
Commune_clean     0.000000
CODGEO_x          0.000000
CODGEO_y          0.000000
CODGEO            0.000000
NBPERS19          0.000000
Q219              0.000000
dtype: float64

### Valeurs manquantes

Les variables de pollution présentent très peu de valeurs manquantes (moins de 0,5 %),
ce qui permet de les exploiter sans précaution particulière.

En revanche, certaines variables socio-économiques issues de Filosofi
(D119 et GI19) comportent environ 57 % de valeurs manquantes.
Cela s’explique par les règles de diffusion de l’INSEE, qui ne publie pas ces indicateurs
pour l’ensemble des communes, notamment lorsque les effectifs sont trop faibles.

Ces valeurs manquantes seront prises en compte dans les analyses ultérieures,
sans imputation arbitraire.


In [35]:
import plotly.express as px

fig = px.histogram(
    df,
    x="pm25",
    nbins=30,
    title="Distribution des concentrations en PM2.5",
    labels={"pm25": "PM2.5 (µg/m³)"}
)
fig.show()


In [ ]:
fig = px.histogram(
    df,
    x="pm10",
    nbins=30,
    title="Distribution des concentrations en PM10",
    labels={"pm10": "PM10 (µg/m³)"}
)
fig.show()

In [ ]:
fig = px.histogram(
    df,
    x="no2",
    nbins=30,
    title="Distribution des concentrations en NO2",
    labels={"no2": "NO2 (µg/m³)"}
)
fig.show()

In [ ]:
fig = px.histogram(
    df,
    x="poverty_index",
    nbins=30,
    title="Distribution des indices de pauvreté",
    labels={"poverty_index": "Indice de pauvreté"}
)
fig.show()

Les distributions des concentrations de PM2.5 et PM10 sont relativement asymétriques à droite,
avec une majorité de communes présentant des niveaux modérés, mais quelques communes plus exposées.
Cela suggère une hétérogénéité spatiale de la pollution en Île-de-France, cohérente avec la présence
de zones urbaines denses et d’axes routiers majeurs.

La distribution du NO2 est plus étalée, ce qui est attendu car ce polluant est fortement lié
au trafic routier, très variable selon les communes.


## Step 2 — Les relations entre les indicateurs de niveau de vie et la pollution


In [ ]:
fig = px.scatter(
    df,
    x="poverty_index",
    y="pm25",
    labels={
        "poverty_index": "Indice de pauvreté",
        "pm25": "PM2.5 (µg/m³)"
    },
    title="PM2.5 et indice de pauvreté par commune"
)

fig.show()



Ce nuage de points met en relation la concentration moyenne en PM2.5 et l’indice de pauvreté
au niveau communal. On observe une légère tendance positive : les communes présentant
un indice de pauvreté plus élevé semblent, en moyenne, légèrement plus exposées aux particules fines.

Cependant, la dispersion reste importante, ce qui indique que le niveau de pollution
n’est pas uniquement déterminé par le niveau de vie. D’autres facteurs, tels que
la densité urbaine, la proximité des axes routiers ou des zones d’activité,
jouent probablement un rôle majeur.

Cette première analyse descriptive suggère l’existence d’inégalités environnementales,
même si celles-ci restent modérées à l’échelle communale.


In [40]:
px.scatter(
    df,
    x="poverty_index",
    y="no2",
    labels={
        "poverty_index": "Indice de pauvreté",
        "no2": "NO2 (µg/m³)"
    },
    title="NO2 et indice de pauvreté par commune"
)


Contrairement aux particules fines, la relation entre le NO2 et l’indice de pauvreté
semble négative. Les communes les plus pauvres présentent, en moyenne,
des concentrations plus faibles de NO2.

Ce résultat s’explique par la nature du NO2, principalement émis par le trafic routier.
Les communes les plus exposées sont généralement les zones urbaines denses
et centrales, qui sont aussi souvent plus favorisées économiquement.
Cette observation est cohérente avec la littérature sur la pollution liée au trafic.


Les nuages de points ne mettent pas en évidence de relation linéaire claire entre pollution et niveau de vie. La forte densité des observations et la faible variance de certains polluants rendent ces graphiques peu lisibles. Afin de mieux comparer les niveaux de pollution entre groupes de communes, nous utilisons ensuite une analyse par quantiles.

In [ ]:
df[["pm25", "pm10", "no2", "poverty_index", "Q219", "GI19"]].corr()

vars_heatmap = ["pm25", "no2", "Q219", "poverty_index", "GI19"]
corr = df[vars_heatmap].corr()

# Palette divergente chaude (orange pastel -> blanc -> rouge)
warm_diverging = [
    [0.0,  "#FFE8C2"],  # orange très clair (corr -1)
    [0.25, "#FFD08A"],  # orange clair
    [0.5,  "#FFF7EC"],  # quasi blanc (corr 0)
    [0.75, "#F59C7D"],  # rouge/orange
    [1.0,  "#B30000"],  # rouge foncé (corr +1)
]

fig = px.imshow(
    corr,
    text_auto=".2f",
    zmin=-1, zmax=1,
    color_continuous_scale=warm_diverging,
    title="Corrélations entre pollution et indicateurs socio-économiques"
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="",
    coloraxis_colorbar=dict(title="Corrélation"),
)

fig.show()



## Step 3 - Analyse descriptive : pollution et indicateurs socio-économiques

Cette section propose une analyse exploratoire du lien entre pollution atmosphérique et niveau de vie au niveau communal en Île-de-France.  
Nous mobilisons deux indicateurs de pollution (PM2.5 et NO2) ainsi que plusieurs indicateurs socio-économiques issus des données INSEE : le revenu médian (Q219), un indice de pauvreté construit à partir des données Filosofi, et l’indice de Gini (GI19), mesurant les inégalités de revenus.


### 1. Corrélations entre polluants

La matrice de corrélation met en évidence une **forte corrélation positive entre les différents polluants**, en particulier entre les concentrations de PM2.5 et de NO2 (corrélation ≈ 0.81).

Ce résultat suggère que ces polluants reflètent en grande partie une **exposition environnementale commune**, probablement liée à des facteurs structurels tels que :
- la densité urbaine,
- l’intensité du trafic routier,
- la concentration des activités économiques.

Ainsi, bien que les polluants aient des origines physiques différentes, ils capturent ici une dimension similaire de la pollution de l’air en milieu urbain.


### 2. Pollution et revenu médian (Q219)

Les corrélations entre la pollution et le revenu médian sont **faibles et négatives** :
- PM2.5 et revenu médian : corrélation ≈ -0.18
- NO2 et revenu médian : corrélation ≈ -0.03

Ces résultats suggèrent qu’en moyenne, les communes ayant un revenu médian plus élevé tendent à être légèrement moins exposées à la pollution aux particules fines, mais que cet effet reste limité.  
Pour le NO2, la corrélation est quasi nulle, ce qui indique que le revenu médian seul ne permet pas d’expliquer l’exposition à une pollution fortement liée au trafic routier.

Une explication plausible est la **structure territoriale de l’Île-de-France** : certaines communes riches peuvent être très centrales et donc fortement exposées au NO2, tandis que des communes moins favorisées peuvent être plus périphériques et moins exposées à ce type de pollution.



### 3. Pollution et indice de pauvreté

L’indice de pauvreté construit présente des **corrélations négatives relativement fortes avec la pollution** :
- PM2.5 et indice de pauvreté : corrélation ≈ -0.59
- NO2 et indice de pauvreté : corrélation ≈ -0.59

À première vue, ce résultat peut sembler contre-intuitif, car une partie de la littérature sur les inégalités environnementales met en avant une surexposition des populations défavorisées à la pollution.

Toutefois, plusieurs éléments invitent à une **interprétation prudente** :
- l’indice de pauvreté utilisé est un **indicateur construit** (ratio entre deux statistiques de niveau de vie), et non un indicateur standardisé ;
- cet indice est t



In [42]:
df["poverty_group"] = pd.qcut(
    df["poverty_index"],
    q=4,
    labels=["Q1 (moins pauvres)", "Q2", "Q3", "Q4 (plus pauvres)"]
)


In [ ]:
px.box(
    df,
    x="poverty_group",
    y="pm25",
    title="PM2.5 selon le niveau de pauvreté des communes"
)

In [44]:
df["income_group"] = pd.qcut(
    df["Q219"],
    q=4,
    labels=["Q1 (revenus faibles)", "Q2", "Q3", "Q4 (revenus élevés)"]
)

px.box(
    df,
    x="income_group",
    y="no2",
    title="NO2 selon le niveau de revenu médian"
)


Le boxplot met en évidence une différence marquée des concentrations de PM2.5 selon le niveau de pauvreté des communes. Les communes les moins pauvres (Q1) présentent en moyenne des niveaux de PM2.5 plus élevés que les communes les plus pauvres (Q4).

Cette observation peut sembler contre-intuitive, mais elle est cohérente avec le fait que les communes les plus favorisées sont souvent plus urbaines et denses, et donc davantage exposées à une pollution de fond liée au trafic et aux activités économiques. À l’inverse, certaines communes plus pauvres peuvent être situées en périphérie ou dans des zones moins denses, avec une exposition moindre aux particules fines.

Ce résultat souligne l’importance de facteurs géographiques et urbains dans l’analyse du lien entre pollution et niveau de vie.

# Step 4 - Relation des indicateurs de niveau de vie et de pollution au niveau infracommunal (carreau 200m*200m)

## Step 1 — Analyse des résultats au niveau infracommunal